In [1]:
from __future__ import absolute_import, unicode_literals, print_function
import numpy as np
import scipy.integrate as integrate
from scipy.constants import pi, hbar, e, m_e, epsilon_0
import pycuba

In [2]:
from __future__ import print_function

import sys

def debug(expression):
    frame = sys._getframe(1)

    print(expression, '=', repr(eval(expression, frame.f_globals, frame.f_locals)))

# Integration with Scipy

In [3]:
# System

V_0 = 1000
debug('V_0')
time_step = 0.25E-15
debug('time_step')

# Fixed shape

R_r = 250.0E-9
debug('R_r')
h_r = 500.0E-9 # [m]
debug('h_r')
h = h_r
debug('h')
d = 1000.0E-9 # [m]
debug('d')
d_plane = d + h_r
debug('d_plane')

xi_r = h_r/d + 1
debug('xi_r')
a = np.sqrt(d**2*R_r**2/(h_r**2+2*d*h_r) + d**2)
debug('a')
eta_1 = - d / a
debug('eta_1')

shift_zd = h - h_r
debug('shift_zd')
max_xi = - (h+d)/(a*eta_1)
debug('max_xi')

theta = np.arccos(d/a)
debug('theta')
r_tip = a*np.sin(theta)*np.tan(theta)
debug('r_tip')
eta_2 = 0.0 # Defines the plane of absorption
debug('eta_2')
eta = eta_1 # Defines the tip
debug('eta')
shift_z = np.abs(a*eta*xi_r)
debug('shift_z')

Q_0 = 1/2*np.log((1+np.cos(theta))/(1-np.cos(theta)))
r = a*np.sin(theta)*np.tan(theta)

# FN
w_theta = 4.7
a_FN = e**2/(16.0*pi**2*hbar)
b_FN = -4.0/(3.0*hbar) * np.sqrt(2.0*m_e*e)
l_const = e / (4.0*pi*epsilon_0)

V_0 = 1000
time_step = 2.5e-16
R_r = 2.5e-07
h_r = 5e-07
h = 5e-07
d = 1e-06
d_plane = 1.5e-06
xi_r = 1.5
a = 1.0246950765959597e-06
eta_1 = -0.9759000729485332
shift_zd = 0.0
max_xi = 1.5
theta = 0.21998797739545933
r_tip = 4.9999999999999945e-08
eta_2 = 0.0
eta = -0.9759000729485332
shift_z = 1.5e-06


In [4]:
def xi_coor(x, y, z):
    res = 1.0/(2.0*a) * ( np.sqrt(x**2 + y**2 + (z + a - shift_z)**2) + np.sqrt(x**2 + y**2 + (z - a - shift_z)**2) )
    return res

def eta_coor(x, y, z):
    res = 1.0/(2.0*a) * ( np.sqrt(x**2 + y**2 + (z + a - shift_z)**2) - np.sqrt(x**2 + y**2 + (z - a - shift_z)**2) )
    return res

def HyperTip_Field_xyz(x, y, z):
    xi = xi_coor(x, y, z)
    eta = eta_coor(x, y, z)
    theta = 0.0
    res = HyperTip_Field(xi, eta, theta)
    return res

def HyperTip_Field(xi, eta, theta):
    ln_fac = np.log((1+eta_1)/(1 - eta_1)*(1-eta_2)/(1+eta_2))
    res = 2*V_0/a * 1.0/(np.sqrt(xi**2 - eta**2)*np.sqrt(1.0 - eta**2)) * 1.0/ln_fac
    return -1.0*res

#field = lambda xi: (V_0/r*np.tan(theta)/(Q_0*np.sqrt(xi**2 - np.cos(theta)**2)))
field = lambda xi: HyperTip_Field(xi, eta_1, 0.0)

In [5]:
print('Field at top')
print(HyperTip_Field(1.0, eta_1, 0.0))
print(field(1.0))

print('Field at bottom')
print(HyperTip_Field(max_xi, eta_1, 0.0))
print(field(max_xi))

Field at top
9301519884.507257
9301519884.507257
Field at bottom
1781848047.5776505
1781848047.5776505


From Fortran program
Top: -876.605169069439       0.000000000000000E+000  -9301519884.50717  
Top: -9301519884.50717  

Bottom: -1706700896.23986       0.000000000000000E+000  -512010268.871958  
Bottom: -1781848047.57765  

In [6]:
def Electron_Supply_old(F):
    J = a_FN/w*F**2
    return time_step*J/e

def Tip_Area(xi_1, xi_2):
    #xi_1 = 1.0
    #xi_2 = max_xi
    fac_1 = xi_1*np.sqrt(xi_1**2 - eta_1**2) - eta_1**2*np.log(xi_1 + np.sqrt(xi_1**2 - eta_1**2))
    fac_2 = xi_2*np.sqrt(xi_2**2 - eta_1**2) - eta_1**2*np.log(xi_2 + np.sqrt(xi_2**2 - eta_1**2))
    area = a**2/2*np.sqrt(1.0 - eta_1**2)*2.0*pi*(fac_2 - fac_1)
    return area

def Tip_Area_2(xi_1, xi_2, phi_1, phi_2):
    fac_1 = xi_1*np.sqrt(xi_1**2 - eta_1**2) - eta_1**2*np.log(xi_1 + np.sqrt(xi_1**2 - eta_1**2))
    fac_2 = xi_2*np.sqrt(xi_2**2 - eta_1**2) - eta_1**2*np.log(xi_2 + np.sqrt(xi_2**2 - eta_1**2))
    res = 0.5*a**2 * np.sqrt(1.0-eta_1**2) * (phi_2 - phi_1) * (fac_2 - fac_1)
    return res

def t_y(F):
    l = l_const * np.abs(F) / w_theta**2 # l = y^2, y = 3.79E-4 * sqrt(F_old) / w_theta
    if ((l > 1.0) or (l < 0.0)):
        print('l = ' + str(l))
    t_y = 1.0 + l*( 1.0/9.0 - 1.0/18.0*np.log(l) )
    #return 1.0
    return t_y

def v_y(F):
    l = l_const * np.abs(F) / w_theta**2 # l = y^2, y = 3.79E-4 * sqrt(F_old) / w_theta
    if ((l > 1.0) or (l < 0.0)):
        print('l = ' + str(l))
    v_y = 1.0 - l + 1.0/6.0 * l * np.log(l)
    #return 1.0
    return v_y

def Elec_supply(F):
    n = a_FN * F**2 * time_step / (e * w_theta * (t_y(F))**2)
    return n

def Escape_Prob(F):
    D_f = np.exp( b_FN * (np.sqrt(w_theta)**3 * v_y(F) / np.abs(F)) )
    return D_f

In [38]:
print('Field at top')
print(HyperTip_Field(1.0, eta_1, 0.0))
print('Supply')
print(Elec_supply(HyperTip_Field(1.0, eta_1, 0.0)))
print('Escape Prob')
print(Escape_Prob(HyperTip_Field(1.0, eta_1, 0.0)))

print('')
print('Field at bottom')
print(HyperTip_Field(max_xi, eta_1, 0.0))
print('Supply')
print(Elec_supply(HyperTip_Field(max_xi, eta_1, 0.0)))
print('Escape Prob')
print(Escape_Prob(HyperTip_Field(max_xi, eta_1, 0.0)))

Field at top
9301519884.507257
Supply
3.766401894211763e+16
Escape Prob
0.07673150170697449

Field at bottom
1781848047.5776505
Supply
1541090101657844.2
Escape Prob
5.165245807739933e-15


In [7]:
print('Tip Area')
print(Tip_Area(1.0, max_xi))
print(Tip_Area_2(1.0, max_xi, 0.0, 2.0*pi))

Tip Area
5.429169853711646e-13
5.429169853711646e-13


Fortran code gives  
Tip Area: 5.429169853668197E-013

In [8]:
int_fun = lambda xi: Elec_supply(field(xi))*Escape_Prob(field(xi))

In [9]:
xi_1 = 1.0
xi_2 = max_xi
result = integrate.quad(int_fun, xi_1, xi_2)

In [10]:
result[0]*Tip_Area(xi_1, xi_2)

9.736993215032351

# From fortran code
4.61890897364688
## Parameters
V             =        1000.00      The Voltage in the system  
d             =       0.150000E-05  Gap spacing in the system  
E_z           =      -0.666667E+09  Z-component of the electric field in the system  
J_FN          =       0.619525E-36  Fowler-Nordheim current density for E_z  
Z_f           =       0.212446E+13  Electron supply  
D_f           =       0.455030E-45  Escape probability  
w_theta       =        4.70000      Work function  
seed          =             2169      The seed used in the random function  

a_foci        =       0.102470E-05  a_foci  
theta_tip     =       0.219988      theta_tip  
r_tip         =       0.500000E-07  r_tip  
eta_1         =      -0.975900      eta_1  
eta_2         =        0.00000      eta_2  
max_xi        =        1.50000      max_xi  
shift_z       =       0.150000E-05  shift_z  

In [18]:
nr_phi = 5000
nr_xi = 5000
len_phi = 2.0*pi
len_xi = max_xi - 1.0

F_avg = 0.0
n_add = 0.0
A_tot = 0.0

for i in range(nr_xi):
    xi_start = 1.0 + i*len_xi/nr_xi
    xi_end = 1.0 + (i+1)*len_xi/nr_xi
    xi_c = (xi_end + xi_start) * 0.5

    for j in range(nr_phi):
        phi_start = j*len_phi/nr_phi
        phi_end = (j+1)*len_phi/nr_phi
        phi_c = (phi_end + phi_start) * 0.5

        #par_pos = xyz_corr(xi_c, eta_1, phi_c)
        F = HyperTip_Field(xi_c, eta_1, phi_c)

        #par_accel = Accel_on_Tip(par_pos)
        #F = Field_normal(par_pos, par_accel) * pre_fac_a

        #if (F < 0.0):
        A_f = Tip_Area_2(xi_start, xi_end, phi_start, phi_end)
        A_tot = A_tot + A_f
        n_add = n_add + A_f*Elec_supply(F) * Escape_Prob(F)
        #print(A_f*Elec_supply(F))
        #print(Escape_Prob(F))
            
debug('n_add')
debug('A_tot')

n_add = 6.369062178369556
A_tot = 5.429169853667627e-13


# Cuba Test

In [37]:
def Integrand_1(ndim, xx, ncomp, ff, userdata):
        # access the current parameters
        xi, phi = [xx[i] for i in range(ndim.contents.value)]
        
        # Transform coodinates from [0, 1]
        phi = phi*2.0*pi
        xi = xi*(max_xi - 1.0) + 1.0
        
        # Calculate scale factors
        h_xi = a*np.sqrt((xi**2 - eta_1**2)/(xi**2 - 1.0))
        h_phi = a*np.sqrt((xi**2 - 1)*(1- eta_1**2))
        
        # Calculate the field
        F = HyperTip_Field(xi, eta_1, phi)
        
        # compute the result
        result = Elec_supply(F) * Escape_Prob(F) * h_xi * h_phi * 2.0*pi*(max_xi - 1.0)
        
        # store the result (here only one component)
        ff[0] = result
        return 0

In [36]:
NDIM = 2
NNEW = 100000
NMIN = 10000
FLATNESS = 50.0
pycuba.Suave(Integrand_1, NDIM, NNEW, NMIN, FLATNESS, verbose=2 | 4)

{'neval': 100000,
 'fail': 1,
 'comp': 0,
 'nregions': 1,
 'results': [{'integral': 6.3672310388145315,
   'error': 0.11510280363696748,
   'prob': -999.0}]}

In [39]:
def xi_coor_xy(x, y):
    res = 1.0/a * 1.0/np.sqrt(1.0 - eta_1**2) * np.sqrt(x**2 + y**2 + a**2*(1.0 - eta_1**2))
    return res

def phi_coor(x, y):
    if ((np.abs(x) < 1.0E-18) and (np.abs(y) < 1.0E-18)):
        res = 0.0
    else:
        res = np.atan2(y, x)
    return res

def Integrand_2(ndim, xx, ncomp, ff, userdata):
        # access the current parameters
        x, y = [xx[i] for i in range(ndim.contents.value)]
        
        x = x*R_r
        y = y*R_r
        
        phi = phi*2.0*pi
        xi = xi*(max_xi - 1.0) + 1.0
        
        # Calculate scale factors
        h_xi = a*np.sqrt((xi**2 - eta_1**2)/(xi**2 - 1.0))
        h_phi = a*np.sqrt((xi**2 - 1)*(1- eta_1**2))
        
        # Calculate the field
        F = HyperTip_Field(xi, eta_1, phi)
        
        # compute the result
        result = Elec_supply(F) * Escape_Prob(F) * h_xi * h_phi
        
        # store the result (here only one component)
        ff[0] = result
        return 0

# Total current from tip

In [42]:
def FN_eq(F):
    res = a_FN * F**2 / (w_theta * (t_y(F))**2) * np.exp( b_FN * (np.sqrt(w_theta)**3 * v_y(F) / np.abs(F)) )
    return res

def Integrand_cur(ndim, xx, ncomp, ff, userdata):
        # access the current parameters
        xi, phi = [xx[i] for i in range(ndim.contents.value)]
        
        # Transform coodinates from [0, 1]
        phi = phi*2.0*pi
        xi = xi*(max_xi - 1.0) + 1.0
        
        # Calculate scale factors
        h_xi = a*np.sqrt((xi**2 - eta_1**2)/(xi**2 - 1.0))
        h_phi = a*np.sqrt((xi**2 - 1)*(1- eta_1**2))
        
        # Calculate the field
        F = HyperTip_Field(xi, eta_1, phi)
        
        # compute the result
        result = FN_eq(F) * h_xi * h_phi * 2.0*pi*(max_xi - 1.0)
        
        # store the result (here only one component)
        ff[0] = result
        return 0

In [46]:
NDIM = 2
NNEW = 100000
NMIN = 10000
FLATNESS = 50.0
results = pycuba.Suave(Integrand_cur, NDIM, NNEW, NMIN, FLATNESS, verbose=2 | 4)

[{'integral': 0.0040805714838483016,
  'error': 7.376600839027278e-05,
  'prob': -999.0}]

In [50]:
results['results'][0]['integral']/1.0E-3

4.080571483848302